## Next Step: 

Conduct with Belmond-Shiba

In [1]:
import pickle
# load
with open('model-kaggle-v0.pkl', 'rb') as f:
    model = pickle.load(f)

In [2]:
talent_list = [
    {
        "name": "アンジュ・カトリーナ",
        "path": "/content/drive/MyDrive/DataScience/chat-ange.npy",
        "color": "#C83C35",
    },
    {
        "name": "リゼ・ヘルエスタ",
        "path": "/content/drive/MyDrive/DataScience/chat-lize.npy",
        "color": "#42FFFF",
    },
    {
        "name": "戌亥とこ",
        "path": "/content/drive/MyDrive/DataScience/chat-toko.npy",
        "color": "#92F3A4",
    }
]

In [3]:
import librosa
audio_filepath = "../data_collection/test-voice/sanbaka-sightseeing.mp4"

# Parameters for mel-spectrogram
sample_rate = 22050  # Standard sample rate for Librosa. Basically equal to sample rate of the audio.
n_mels = 128         # Number of mel bands
n_fft = 2048         # Length of the FFT window -> ブログのやつでは173(分解能を11.5ms にするため！)
hop_length = 256     # Number of samples between successive frames
win_length = n_fft     # window length
segment_duration = 2  # seconds

y, sr = librosa.load(audio_filepath, sr = sample_rate)

/var/folders/n9/f02ch2416_7cp5wybnwwhh7r0000gn/T/ipykernel_4216/3694596002.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_filepath, sr = sample_rate)
/opt/miniconda3/envs/general/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [5]:
# Kaggle code (https://www.kaggle.com/code/anmour/svm-using-mfcc-features)

import numpy as np
import pandas as pd

import os
import librosa

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

# Generate mfcc features with mean and standard deviation
def get_mfcc(segment, SAMPLE_RATE):
    data = segment
    # try:

    ft1 = librosa.feature.mfcc(y = data, sr = SAMPLE_RATE, n_mfcc=30)
    ft2 = librosa.feature.zero_crossing_rate(y = data)[0]
    ft3 = librosa.feature.spectral_rolloff(y = data, sr = SAMPLE_RATE)[0]
    ft4 = librosa.feature.spectral_centroid(y = data, sr = SAMPLE_RATE)[0]
    ft5 = librosa.feature.spectral_contrast(y = data, sr = SAMPLE_RATE)[0]
    ft6 = librosa.feature.spectral_bandwidth(y = data, sr = SAMPLE_RATE)[0]
    ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
    ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
    ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
    ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
    ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
    ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
    return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    
    # except:
    #     print('bad file')
    #     return pd.Series([0]*210)

In [7]:
import pandas as pd
import numpy as np
import librosa

diarization = pd.read_csv("../pyannote/result-sightseeing.csv")

labels = [d["name"] for d in talent_list]
colors = [d["color"] for d in talent_list]

# Example entries
srt_elements = [
    # {'start': 1.0, 'end': 4.0, 'text': 'This is red text', 'color': '#FF0000'},
    # {'start': 5.0, 'end': 7.0, 'text': 'This is green text', 'color': '#00FF00'},
    # {'start': 6.5, 'end': 10.0, 'text': 'This is blue text overlapping', 'color': '#0000FF'},
]

for index, row in diarization.iterrows():
    print(f"{row["start"]} - {row["end"]}: {row["speaker"]} said, ''{row["text"]}''")
    start = max(int(sr * row["start"]), 0)
    end = min(int(sr * row["end"]), len(y))
    turn = y[start:end]

    turn_len = len(turn)
    # 0-2, 1-3, 2-4,,,
    num_segments = int( turn_len / sr ) #  - segment_duration + 1
    mfccs = []
    for segment_index in range(num_segments):
        segment = y[segment_index * sr : (segment_index + 1 ) * sr ]
        # (segment_index + segment_duration) * sr ]

        segment = np.concatenate((segment, segment), axis = 0)

        # mfcc = np.mean(librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)[1:], axis=1)
        mfcc = get_mfcc(segment, 22050)
        if np.isnan(mfcc).any():
            # if mfcc array contain nan value, skip
            continue
        
        mfccs.append(mfcc)

    if len(mfccs) == 0:
        print("Turn's duration is under 2 sec. Unable to evaluate")
        continue

    # change to PCA
    # Apply scaling for PCA
    scaler = StandardScaler()
    mfccs_scaled = scaler.fit_transform(mfccs)
    # Apply PCA for dimension reduction
    import pickle
    pca = pickle.load(open("pca.pkl",'rb'))
    mfccs_pca = pca.transform(mfccs_scaled)
    
    predictions = model.predict(mfccs_pca)
    print(predictions)

    values, counts = np.unique(predictions, return_counts=True)
    print(values)
    print(counts)

    print(f"Most Dominant: {values[np.argmax(counts)]}")

    color = colors[np.argmax(counts)]

    srt_elements.append({
        'start': row["start"],
        'end': row["end"],
        'text': row["text"],
        'color': color
    })



0.03096875 - 5.53221875: SPEAKER_01 said, ''音量調整したけど視聴者さん側にも念のため聞いてもらおうかにしましょう''


ValueError: X has 65 features, but SVC is expecting 210 features as input.

In [9]:
def format_time(secs: float) -> str:
    """Convert seconds as float to SRT time format (hh:mm:ss,ms)."""
    hours =  int(secs) // 3600
    minutes = (int(secs) % 3600) // 60
    seconds = ( int(secs) % 60 ) // 1
    milliseconds = int((secs - (int(secs) // 1)) * 1000 )
    return f"{str(hours).zfill(2)}:{str(minutes).zfill(2)}:{str(seconds).zfill(2)},{str(milliseconds).zfill(3)}"

def create_srt_entry(index: int, start_time: float, end_time: float, text: str, color_code: str) -> str:
    """Create a formatted SRT entry."""
    start_time_str = format_time(start_time)
    end_time_str = format_time(end_time)
    formatted_text = f"[color={color_code}]{text}[/color]"
    return f"{index}\n{start_time_str} --> {end_time_str}\n{formatted_text}\n"

def write_srt_file(entries: list, output_file: str):
    """Write entries to an SRT file."""
    with open(output_file, 'w', encoding='utf-8') as file:
        for i, entry in enumerate(entries, start=1):
            file.write(create_srt_entry(i, entry['start'], entry['end'], entry['text'], entry['color']))
            file.write("\n")

# Write to an SRT file
write_srt_file(srt_elements, 'sanbaka-sightseeing.srt')
